In [19]:
from typing import Tuple
from sklearn.cluster import AgglomerativeClustering
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import pathlib
import pandas as pd
import numpy as np
import csv
n_clusters = 4

**Bases de datos**

1. EVA 
2. PROYECTOS_HISTORICO (Hisotrico de proyectos completo.csv)

**Registro de usuario**

1. Cadena productiva CP
2. Departamento DP
3. Total de beneficiarios BN
4. Valor cofinanciación VC

In [22]:
#Traer de la base de datos DIFICIL
PATH = pathlib.Path('./data').parent
DATA_PATH = PATH.joinpath("data").resolve()
EVA = pd.read_csv(DATA_PATH.joinpath('EVA_cultivos_modificado.csv'))
PROYECTOS_HISTORICO= pd.read_csv('Hisotrico de proyectos completo.csv')

In [13]:
registro = ['COCO', 'ANTIOQUIA',11,2e6]

In [16]:
#info_adicional = EVA.groupby(['DEPARTAMENTO','CADENA_PRODUCTIVA_ADR','CICLO_CULTIVO'])[['RENDIMIENTO','PRODUCTIVIDAD']].mean().reset_index()

In [23]:
#Transformar data historica
def transform_hist(X1):
    
    scaler = StandardScaler()
    dff= X1.drop(columns = ['DEPARTAMENTO','MUNICIPIO', 'YEAR','CP','CICLO_CULTIVO']).copy()
    scaler.fit(dff)
    X2 = scaler.transform(dff)
    X3 =pd.DataFrame(X2, columns = dff.columns)
    X4 = pd.merge(X3,X1[['CP','CICLO_CULTIVO']],right_index=True,left_index=True)
    
    #Creating dummy variables
    cc_dummy = pd.get_dummies(X4['CICLO_CULTIVO'])
    cp_dummy = pd.get_dummies(X4['CP'])
    #Concatenating the dummy variables to the original dataset 
    cluster_dummy_set=pd.concat([X4,cc_dummy,cp_dummy],axis=1)
    #Deleting categorical variable from the dummy set
    del cluster_dummy_set['CICLO_CULTIVO']
    del cluster_dummy_set['CP']
    
    MCU = cluster_dummy_set.sample(frac = 0.5,random_state=42)
    MCL= cluster_dummy_set[~cluster_dummy_set.index.isin(MCU.index)]
    
    MCU=MCU.dropna()
    MCL=MCL.dropna()
    
    return MCU,MCL

In [8]:
def get_clusters(X_train: pd.DataFrame, X_test: pd.DataFrame, n_clusters: int) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    applies k-means clustering to training data to find clusters and predicts them for the test set
    """
    clustering = AgglomerativeClustering(n_clusters=n_clusters_, linkage=
                                     'ward').fit(X_train)
    # apply the labels
    train_labels = clustering.labels_
    X_train_clstrs = X_train.copy()
    X_train_clstrs['clusters'] = train_labels
    
    # predict labels on the test set
    test_labels = clustering.predict(X_test)
    X_test_clstrs = X_test.copy()
    X_test_clstrs['clusters'] = test_labels
    return X_train_clstrs, X_test_clstrs

In [9]:
CALIFICACION = np.array([[2, 1, 4, 3],[3, 4, 1, 2],[1, 2, 3, 1],[4, 3, 2, 4]])
PESOS = np.array([[0.3],[0.1],[0.2],[0.4]])

In [ ]:
# Crear la calificacion
def calificar(X_train_clstrs, X1):
    idx = X_train_clstrs.index
    X5 = X1.iloc[idx].copy()
    X5['key'] = idx
    

In [ ]:
# Transformar datos usuario
def transform_input():
    
    
    return DU

In [10]:
#Clasificacion

def get_classifiers (n_clusters, X1, DU, CALIFICACION, PESOS):
    
    MCU, MCL = transform_hist(X1)
    X_train_clstrs, X_test_clstrs = get_clusters(MCU, MCL, n_clusters)
    X = X_test_clstrs.drop(columns = ['clusters'])
    y = X_test_clstrs['clusters']
    rf = RandomForestClassifier(n_estimators=10)
    rf.fit(X, y)
    y_pred = rf.predict(DU)
    y_prob = rf.predict_proba(DU) 
    APROBADO = CALIFICACION.dot(PESOS)/4
    PROBABILIDAD_APROBADO = y_prob*APROBADO.T
    
    return PROBABILIDAD_APROBADO.max()*100